In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementNotInteractableException
import time
from tqdm import tqdm
import csv
from selenium.webdriver.chrome.options import Options

# run Chrome in headless mode
options = Options()

# disable Blink to disallow images
options.add_argument("--blink-settings=imagesEnabled=false")

# set the options to use Chrome in headless mode (doesnt open browser)
options.add_argument("--headless=new")


# Initialize the WebDriver
service = Service(executable_path='C:/Users/mhand/Selenium/chromedriver-win64/chromedriver.exe')
driver = webdriver.Chrome(service=service, options=options)

# Open the page containing the table of players
driver.get('https://www.futbolfantasy.com/analytics/laliga-fantasy/puntos')

driver.execute_script("document.body.style.zoom='33%'")

def accept_cookies(driver):
        """Clicks on accept in the cookies pop-up"""
        try:
            # Wait for the cookie banner to be present (adjust time if necessary)
            cookies_xpath = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'  # Change this if necessary
            WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, cookies_xpath)))
            
            # Click the "Accept Cookies" button
            accept_cookies = driver.find_element(By.XPATH, cookies_xpath)
            accept_cookies.click()
            print("Cookies accepted successfully.")
        except TimeoutException:
            print("Cookie banner did not appear.")
        except Exception as e:
            print(f"An error occurred while accepting cookies: {e}")

accept_cookies(driver)

player_data_list = []

player_divs = driver.find_elements(By.CLASS_NAME, 'elemento_jugador')

# Iterate through each player's div
for i, player in enumerate(tqdm(player_divs, desc="Processing Players", unit="player")):
    try:
        player_data = {}

        # Get player link from the main page
        player_data['name'] = player.get_attribute('data-nombre')
        player_data['link'] = 'https://www.futbolfantasy.com/jugadores/'+player_data['name'].replace(' ', '-')

        player_data_list.append(player_data)

        # Wait for the player list to reload before the next iteration
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'lista_elementos')))

        # Re-fetch the player divs (avoid stale elements error)
        player_divs = driver.find_elements(By.CLASS_NAME, 'elemento_jugador')

    except TimeoutException:
        print("Timeout occurred while processing player:", player_data.get('name', 'Unknown'))
        
    except ElementNotInteractableException:

        next_button = driver.find_element(By.CSS_SELECTOR, '.next')
        next_button.click()
        player_data = {}

        # Get player link from the main page
        player_data['name'] = player.get_attribute('data-nombre')
        player_data['link'] = 'https://www.futbolfantasy.com/jugadores/'+player_data['name'].replace(' ', '-')

        player_data_list.append(player_data)

        # Wait for the player list to reload before the next iteration
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'lista_elementos')))

        # Re-fetch the player divs (avoid stale elements error)
        player_divs = driver.find_elements(By.CLASS_NAME, 'elemento_jugador')

# Save data to CSV
csv_file = "player_links.csv"
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=player_data_list[0].keys())
    writer.writeheader()
    writer.writerows(player_data_list)

print(f"Links saved to {csv_file}")

driver.quit()


Cookies accepted successfully.


Processing Players: 100%|██████████| 752/752 [01:11<00:00, 10.47player/s]


Data saved to player_data.csv


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementNotInteractableException
import time
from tqdm import tqdm
import csv
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup


In [2]:
def accept_cookies(driver):
        """Clicks on accept in the cookies pop-up"""
        try:
            # Wait for the cookie banner to be present (adjust time if necessary)
            cookies_xpath = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'  # Change this if necessary
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, cookies_xpath)))
            
            # Click the "Accept Cookies" button
            accept_cookies = driver.find_element(By.XPATH, cookies_xpath)
            accept_cookies.click()
            print("Cookies accepted successfully.")
        except TimeoutException:
            print("Cookie banner did not appear.")
        except Exception as e:
            print(f"An error occurred while accepting cookies: {e}")

In [102]:
# import csv

# # run Chrome in headless mode
# options = Options()

# # disable Blink to disallow images
# options.add_argument("--blink-settings=imagesEnabled=false")

# # set the options to use Chrome in headless mode (doesnt open browser)
# # options.add_argument("--headless=new")

# # Initialize the WebDriver
# service = Service(executable_path='C:/Users/mhand/Selenium/chromedriver-win64/chromedriver.exe')
# driver = webdriver.Chrome(service=service, options=options)

# with open('player_links.csv', newline='') as csvfile:

#     reader = csv.reader(csvfile, delimiter=',', quotechar='|')

#     next(reader)

#     for i, link in enumerate(tqdm(reader)):
#         time.sleep(1)

#         player_data = {}

#         driver.get(link[1])

#         if i == 0:
#             accept_cookies(driver)
        
#         player_data['XPlay'] = driver.find_element(By.CSS_SELECTOR, 'div.pb-2:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(2)').text
#         # player_data['Value'] = driver.find_element(By.CSS_SELECTOR, '.valor-actual').text


#         # table = driver.find_element(By.CSS_SELECTOR, 'table.tablestats:nth-child(5) > tbody:nth-child(2)')
#         table = driver.find_element(By.CLASS_NAME, 'tablestats')

#         rows = table.find_elements(By.XPATH, '/html/body/div[1]/div/div[1]/main/div[2]/section/div/div[2]/div/div[6]/div/div[3]/table/tbody/tr[1]')
#         print(len(rows))

#         for row in rows:
#             GW = row.find_element(By.CLASS_NAME, 'bold jorn-td').text
#             player_data['Points_'+GW] = row.find_element(By.CLASS_NAME, 'data points bold d-flex').text

#         print(player_data)
        


# driver.close()

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv
from tqdm import tqdm

data = []
bad_links = []
other_errors = []

with open('player_links.csv', newline='') as csvfile:
        
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')

    next(reader)

    for i, link in enumerate(tqdm(reader, total=752)):

        try:
            
            player_data = {}

            response = requests.get(link[1], timeout=10)
            response.raise_for_status()


            soup = BeautifulSoup(response.content, 'html.parser')

            XPlay = soup.find('div', class_='pct').get_text()
            name = soup.select('span.d-none.d-lg-inline-block')[0].get_text()
            team_name = soup.select_one('div.img-underphoto a').get_text().strip()

            player_data['Name'] = name
            player_data['TeamName'] = team_name
            player_data['XPlay'] = XPlay


            point_boxes = soup.select('span.racha-box.columna_puntos.point.mx-auto.laliga-fantasy')

            points = []

            for box in point_boxes:
                text = box.get_text()
                p = re.findall(r'-?\d+', text)
                points.extend(p)

            table = soup.find('table', class_='tablestats')
            rows = table.find_all('tr', class_='plegado plegable')

            for i, row in enumerate(rows):
                GW = row.find('td', class_=re.compile(r'bold')).get_text()
                # print(f'Gameweek {GW}: {points[i]}')
                player_data['Points_GW'+GW] = points[i]

            data.append(player_data)

        except requests.exceptions.RequestException as e:
            # print(f"Error fetching {link[1]}: {e}")
            bad_links.append(link)
        except Exception as e:
            # print(f"An error occurred with {link[1]}: {e}")
            other_errors.append(link)

100%|██████████| 752/752 [04:23<00:00,  2.85it/s]


In [2]:
import pandas as pd

df = pd.DataFrame(data)

In [3]:
df[df['TeamName'] == 'Betis']

,Name,TeamName,XPlay,Points_GW8,Points_GW7,Points_GW6,Points_GW5,Points_GW4,Points_GW3,Points_GW2,Points_GW1
49,Rui Silva,Betis,95%,10,5,5,9,3,5,11,3
50,Marc Roca,Betis,80%,4,3,7,7,3,11,4,4
59,Abde Ezzalzouli,Betis,80%,5,4,4,11,4,8,3,4
64,Giovani Lo Celso,Betis,95%,17,15,9,3,NaN,21,NaN,NaN
103,Sergi Altimira,Betis,70%,4,0,11,8,NaN,5,NaN,3
126,Pablo Fornals,Betis,70%,3,5,3,8,1,2,7,3
127,Diego Javier Llorente,Betis,90%,10,5,2,6,1,3,9,3
144,Youssouf Sabaly,Betis,0%,NaN,NaN,NaN,7,3,NaN,8,4
155,Marc Bartra,Betis,30%,NaN,NaN,NaN,NaN,NaN,NaN,8,13
160,Natan,Betis,80%,8,7,4,10,2,4,0,NaN


In [4]:
df.to_csv('data/players.csv', index=False)

In [5]:
import requests
from bs4 import BeautifulSoup

# Send a GET request to the webpage URL
url = 'https://www.futbolfantasy.com/jugadores/adrian-san-miguel'

response = requests.get(url)

# Parse the content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

point_boxes = soup.select('span.racha-box.columna_puntos.point.mx-auto.laliga-fantasy')

points = []

for box in point_boxes:
    text = box.get_text()
    p = re.findall(r'-?\d+', text)
    points.extend(p)

print(points)

['-1']


In [6]:
bad_links

[['eder militao', 'https://www.futbolfantasy.com/jugadores/eder-militao'],
 ['iÃ±aki williams',
  'https://www.futbolfantasy.com/jugadores/iÃ±aki-williams'],
 ['samu costa', 'https://www.futbolfantasy.com/jugadores/samu-costa'],
 ['iÃ±igo martinez',
  'https://www.futbolfantasy.com/jugadores/iÃ±igo-martinez'],
 ['Ã–rjan nyland', 'https://www.futbolfantasy.com/jugadores/Ã–rjan-nyland'],
 ['andrei ratiu', 'https://www.futbolfantasy.com/jugadores/andrei-ratiu'],
 ['oscar mingueza', 'https://www.futbolfantasy.com/jugadores/oscar-mingueza'],
 ['oscar valentin', 'https://www.futbolfantasy.com/jugadores/oscar-valentin'],
 ['alex baena', 'https://www.futbolfantasy.com/jugadores/alex-baena'],
 ['beÃ±at turrientes',
  'https://www.futbolfantasy.com/jugadores/beÃ±at-turrientes'],
 ['jose angel carmona',
  'https://www.futbolfantasy.com/jugadores/jose-angel-carmona'],
 ['santi comesaÃ±a',
  'https://www.futbolfantasy.com/jugadores/santi-comesaÃ±a'],
 ['beÃ±at prados', 'https://www.futbolfantasy.co

In [7]:
len(other_errors)

227

In [4]:
players = pd.read_csv('data/players.csv')
matches = pd.read_csv('data/matches.csv')

In [5]:
players = players.replace('Athletic', 'Athletic Club')
players = players.replace('Atlético', 'Atlético Madrid')
players = players.replace('Celta', 'Celta Vigo')
players = players.replace('Rayo', 'Rayo Vallecano')

In [6]:
import json

with open('club_ids.json', 'r') as fp:
        club_mapping = json.load(fp)

In [7]:
players['TeamId'] = players['TeamName'].map(club_mapping)

In [8]:
home_matches_by_player = pd.merge(left=matches, right=players, left_on='Home_id', right_on='TeamId', how='left')

In [9]:
away_matches_by_player = pd.merge(left=matches, right=players, left_on='Away_id', right_on='TeamId', how='left')

In [10]:
matches_by_player = pd.concat([home_matches_by_player, away_matches_by_player]).drop(['Time', 'Date', 'xG', 'xG.1', 'Day', 'Score', 'XPlay'], axis=1)

In [11]:
matches_by_player

,Wk,Home,Away,Home_id,Away_id,Name,TeamName,Points_GW8,Points_GW7,Points_GW6,Points_GW5,Points_GW4,Points_GW3,Points_GW2,Points_GW1,TeamId
0,1.0,Athletic Club,Getafe,2,8,Dani Vivian,Athletic Club,NaN,18.0,7.0,4.0,4.0,12.0,4.0,2.0,2
1,1.0,Athletic Club,Getafe,2,8,Oihan Sancet,Athletic Club,NaN,2.0,5.0,8.0,5.0,7.0,10.0,9.0,2
2,1.0,Athletic Club,Getafe,2,8,Gorka Guruzeta,Athletic Club,3.0,6.0,16.0,3.0,4.0,4.0,2.0,7.0,2
3,1.0,Athletic Club,Getafe,2,8,Julen Agirrezabala,Athletic Club,5.0,13.0,0.0,3.0,4.0,9.0,NaN,NaN,2
4,1.0,Athletic Club,Getafe,2,8,Álex Berenguer,Athletic Club,5.0,6.0,6.0,3.0,3.0,NaN,3.0,3.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8678,38.0,Rayo Vallecano,Mallorca,14,12,Iván Cuéllar,Mallorca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
8679,38.0,Rayo Vallecano,Mallorca,14,12,Daniel Luna,Mallorca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
8680,38.0,Rayo Vallecano,Mallorca,14,12,Marc Doménech,Mallorca,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,12
8681,38.0,Rayo Vallecano,Mallorca,14,12,Robert Navarro,Mallorca,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,12


In [12]:
import pandas as pd
import numpy as np

class Formatting:
    def __init__(self, dataframe: pd.DataFrame):
        self.dataframe = dataframe.copy()

    def __target_and_previous(self):
        """Prepares the target column as the player performance in points as well as the last three performances."""
        # Initialize the new columns
        self.dataframe['Target'] = np.nan
        self.dataframe['Points_Tmin1'] = np.nan
        self.dataframe['Points_Tmin2'] = np.nan
        self.dataframe['Points_Tmin3'] = np.nan

        for wk in self.dataframe['Wk'].unique():
            wk = int(wk)

            # Check if Points_GW{wk} column exists and assign 'Target'
            target_column = f'Points_GW{wk}'

            if target_column in self.dataframe.columns:
                # Assign values to the 'Target' column for all rows where Wk == wk
                self.dataframe.loc[self.dataframe['Wk'] == wk, 'Target'] = self.dataframe.loc[self.dataframe['Wk'] == wk, target_column].values

            # else:
            #     self.dataframe = self.dataframe[self.dataframe['Wk'] != wk]

            # Assign Tmin values based on prior GWs
            if wk - 1 > 0 and f'Points_GW{wk-1}' in self.dataframe.columns:
                self.dataframe.loc[self.dataframe['Wk'] == wk, 'Points_Tmin1'] = self.dataframe.loc[self.dataframe['Wk'] == wk, f'Points_GW{wk-1}'].values
            if wk - 2 > 0 and f'Points_GW{wk-2}' in self.dataframe.columns:
                self.dataframe.loc[self.dataframe['Wk'] == wk, 'Points_Tmin2'] = self.dataframe.loc[self.dataframe['Wk'] == wk, f'Points_GW{wk-2}'].values
            if wk - 3 > 0 and f'Points_GW{wk-3}' in self.dataframe.columns:
                self.dataframe.loc[self.dataframe['Wk'] == wk, 'Points_Tmin3'] = self.dataframe.loc[self.dataframe['Wk'] == wk, f'Points_GW{wk-3}'].values

        # Drop all 'Points_GW' columns
        gw_columns = [col for col in self.dataframe.columns if col.startswith('Points_GW')]
        self.dataframe.drop(gw_columns, axis=1, inplace=True)

    def __home_dummy(self):
        """Sets column Home to 1 if the player belongs to the team playing at home, else 0."""

        self.dataframe['Home?'] = np.where(self.dataframe['Home_id'] == self.dataframe['TeamId'], 1, 0)

        self.dataframe['Opponent'] = np.where(self.dataframe['Home?'] == 1, self.dataframe['Away'], self.dataframe['Home'])
        self.dataframe['Opponent_id'] = np.where(self.dataframe['Home?'] == 1, self.dataframe['Away_id'], self.dataframe['Home_id'])
        self.dataframe.drop(['Away_id', 'Home_id', 'Away', 'Home'], axis=1, inplace=True)

    def __reorder(self):
        """Sets final ordering of columns."""

        final_order = ['Wk', 'Name', 'TeamName', 'Home?', 'TeamId', 'Opponent', 'Opponent_id', 'Points_Tmin1', 'Points_Tmin2', 'Points_Tmin3', 'Target']
        self.dataframe = self.dataframe[final_order]


    def preprocess(self):
        """Returns dataframe upon applying all preprocessing steps."""

        self.__target_and_previous()

        self.__home_dummy()

        self.__reorder()

        return self.dataframe


In [13]:
formatter = Formatting(matches_by_player)

df1 = formatter.preprocess()

In [14]:
df1_clean = df1.fillna(value=-5).drop(['Opponent', 'TeamName'], axis=1)

In [15]:
df1_clean = df1_clean[df1_clean['Wk'] > 3]

In [16]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()

df1_clean['Name'] = enc.fit_transform(df1_clean['Name'])

In [17]:
df1_clean

,Wk,Name,Home?,TeamId,Opponent_id,Points_Tmin1,Points_Tmin2,Points_Tmin3,Target
662,4.0,264,1,4,19,12.0,10.0,12.0,15.0
663,4.0,369,1,4,19,2.0,10.0,15.0,17.0
664,4.0,360,1,4,19,8.0,12.0,9.0,26.0
665,4.0,348,1,4,19,12.0,7.0,2.0,6.0
666,4.0,250,1,4,19,3.0,4.0,2.0,14.0
...,...,...,...,...,...,...,...,...,...
8678,38.0,186,0,12,14,-5.0,-5.0,-5.0,-5.0
8679,38.0,115,0,12,14,-5.0,-5.0,-5.0,-5.0
8680,38.0,288,0,12,14,-5.0,-5.0,-5.0,-5.0
8681,38.0,370,0,12,14,-5.0,-5.0,-5.0,-5.0


In [18]:
X_train = df1_clean[df1_clean['Wk'] < 9].drop(['Target', 'Wk'], axis=1) 
X_test = df1_clean[df1_clean['Wk'] == 9].drop(['Target', 'Wk'], axis=1)
y = df1_clean[df1_clean['Wk'] < 9]['Target']

In [19]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators=200, max_depth=None, random_state=0)
regr.fit(X_train, y)

RandomForestRegressor(n_estimators=200, random_state=0)

In [20]:
preds = regr.predict(X_test)

In [21]:
regr.feature_importances_

array([0.21017212, 0.02372452, 0.11053434, 0.11922218, 0.27634525,
       0.15678105, 0.10322052])

In [22]:
GW9_prediction = pd.DataFrame({'Name':enc.inverse_transform(X_test['Name']), 'Prediction':preds})

In [26]:
GW9_prediction.sort_values(by='Prediction', ascending=False).head(30)

,Name,Prediction
412,Giovani Lo Celso,9.535
117,Rodrygo Goes,7.750
385,Robert Lewandowski,7.510
324,Sergio Camello,7.005
318,Abdul Mumin,6.935
246,Antonio Raíllo,6.460
328,Augusto Batalla,6.440
388,Jules Koundé,6.335
320,Florian Lejeune,6.210
0,Seydouba Cisse,6.190
